In [1]:
kaggle_env = False

if not kaggle_env:
  from google.colab import drive
  drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
if not kaggle_env:
  !pip install kaggle --upgrade

  import os
  import json
  f = open("./drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
  json_data = json.load(f)
  os.environ["KAGGLE_USERNAME"] = json_data["username"]
  os.environ["KAGGLE_KEY"] = json_data["key"]

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [3]:
if not kaggle_env:
  # Change current directory
  %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative


In [4]:
if not kaggle_env:
  # Download Kaggle datasets related to this notebook
  #!kaggle datasets list -s shahules/ner-coleridge-initiative
  !mkdir input/ner-coleridge-initiative
  !kaggle datasets download shahules/ner-coleridge-initiative -p input/ner-coleridge-initiative
  #!ls input/ner-coleridge-initiative
  !unzip input/ner-coleridge-initiative/ner-coleridge-initiative.zip -d input/ner-coleridge-initiative
  !rm input/ner-coleridge-initiative/ner-coleridge-initiative.zip
  #!ls input/ner-coleridge-initiative

mkdir: cannot create directory ‘input/ner-coleridge-initiative’: File exists
 95% 120M/126M [00:00<00:00, 148MB/s]
100% 126M/126M [00:00<00:00, 136MB/s]


In [4]:
if not kaggle_env:
  %cd notebooks

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks


In [6]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

if kaggle_env:
    import nltk
    import spacy
    nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
    nlp.max_length = 4000000
    from nltk.probability import FreqDist
    from wordcloud import WordCloud, STOPWORDS

from tqdm.autonotebook import tqdm
import string

%matplotlib inline

if kaggle_env:
    os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')
else:
    os.listdir('../input/coleridgeinitiative-show-us-the-data/')

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

In [9]:
# reading csv files and train & test file paths
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [10]:
if kaggle_env:
    dirname = "/kaggle/input/coleridgeinitiative-show-us-the-data"
else:
    dirname = "../input/coleridgeinitiative-show-us-the-data"
#df = pd.read_csv(os.path.join(dirname, "train.csv"))

In [11]:
# Credit to: https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230341
df = pd.read_pickle("../input/ner-coleridge-initiative/show_us_the_data_train_ner.pkl")

In [12]:
df

,token,sentence#,sentence,entity
0,This,sentence#0,sentence0,o
1,study,sentence#0,sentence0,o
2,used,sentence#0,sentence0,o
3,data,sentence#0,sentence0,o
4,from,sentence#0,sentence0,o
...,...,...,...,...
53139372,to,sentence#15,sentence9999,o
53139373,many,sentence#15,sentence9999,o
53139374,other,sentence#15,sentence9999,o
53139375,biological,sentence#15,sentence9999,o


In [13]:
df["token"].nunique()

857922

In [14]:
df["sentence#"].value_counts(dropna=False)

sentence#0      2899042
sentence#1      2892327
sentence#2      2876816
sentence#3      2852276
sentence#4      2827468
                 ...   
sentence#209        290
sentence#201        290
sentence#236        290
sentence#223        290
sentence#242         72
Name: sentence#, Length: 243, dtype: int64

In [15]:
df["sentence#"].unique()

array(['sentence#0', 'sentence#1', 'sentence#2', 'sentence#3',
       'sentence#4', 'sentence#5', 'sentence#6', 'sentence#7',
       'sentence#8', 'sentence#9', 'sentence#10', 'sentence#11',
       'sentence#12', 'sentence#13', 'sentence#14', 'sentence#15',
       'sentence#16', 'sentence#17', 'sentence#18', 'sentence#19',
       'sentence#20', 'sentence#21', 'sentence#22', 'sentence#23',
       'sentence#24', 'sentence#25', 'sentence#26', 'sentence#27',
       'sentence#28', 'sentence#29', 'sentence#30', 'sentence#31',
       'sentence#32', 'sentence#33', 'sentence#34', 'sentence#35',
       'sentence#36', 'sentence#37', 'sentence#38', 'sentence#39',
       'sentence#40', 'sentence#41', 'sentence#42', 'sentence#43',
       'sentence#44', 'sentence#45', 'sentence#46', 'sentence#47',
       'sentence#48', 'sentence#49', 'sentence#50', 'sentence#51',
       'sentence#52', 'sentence#53', 'sentence#54', 'sentence#55',
       'sentence#56', 'sentence#57', 'sentence#58', 'sentence#59',
     

In [16]:
df["sentence"].value_counts(dropna=False)

sentence6030    70252
sentence2770    57077
sentence950     57077
sentence7045    39753
sentence3418    38432
                ...  
sentence3724      213
sentence8659      213
sentence3793       63
sentence8653       63
sentence8652       57
Name: sentence, Length: 10000, dtype: int64

In [17]:
df["sentence"].unique()

array(['sentence0', 'sentence1', 'sentence2', ..., 'sentence9997',
       'sentence9998', 'sentence9999'], dtype=object)

In [18]:
pd.options.display.max_rows = 60 #9999

In [19]:
df[df["sentence"]=="sentence9999"]

,token,sentence#,sentence,entity
53134924,Motivation:,sentence#0,sentence9999,o
53134925,The,sentence#0,sentence9999,o
53134926,identification,sentence#0,sentence9999,o
53134927,of,sentence#0,sentence9999,o
53134928,quantitative,sentence#0,sentence9999,o
...,...,...,...,...
53139372,to,sentence#15,sentence9999,o
53139373,many,sentence#15,sentence9999,o
53139374,other,sentence#15,sentence9999,o
53139375,biological,sentence#15,sentence9999,o


In [20]:
df[df["sentence"]=="sentence0"]

,token,sentence#,sentence,entity
0,This,sentence#0,sentence0,o
1,study,sentence#0,sentence0,o
2,used,sentence#0,sentence0,o
3,data,sentence#0,sentence0,o
4,from,sentence#0,sentence0,o
...,...,...,...,...
1759,(version,sentence#6,sentence0,o
1760,2.1),sentence#6,sentence0,o
1761,for,sentence#6,sentence0,o
1762,additional,sentence#6,sentence0,o


#### Data Preparation for NERD fine-tuning
- data structure:  
    {'sentences': [[], [], ..., []],  
     'tags': [[], [], ..., []]}

In [ ]:
#tmp = df.groupby("sentence")["entity"].count()

In [21]:
# CV (GroupKfold)
"""example
import numpy as np
from sklearn.model_selection import GroupKFold
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

print(group_kfold)

for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)
"""

X = df.index.values
y = df["entity"].values
groups = df["sentence"].values

group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(X, y, groups)

print(group_kfold)

res = {}
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, groups)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    res[i] = [X_train, y_train, X_test, y_test]

GroupKFold(n_splits=5)


In [ ]:
# ToDo: save res as a pickle file

In [23]:
res[0]

[array([       0,        1,        2, ..., 53139374, 53139375, 53139376]),
 array(['o', 'o', 'o', ..., 'o', 'o', 'o'], dtype=object),
 array([    1764,     1765,     1766, ..., 53134921, 53134922, 53134923]),
 array(['o', 'o', 'o', ..., 'o', 'o', 'o'], dtype=object)]

In [ ]:
# train / dev split
#train, dev = train_test_split(df, test_size=0.3, random_state=42, shuffle=True, stratify=df["sentence"])

In [26]:
# CV 1
i = 0
X_train = res[i][0]
y_train = res[i][1]
X_test = res[i][2]
y_test = res[i][3]

In [ ]:
df[df.index.isin(X_train)]